In [ ]:
%load_ext autoreload
%autoreload 2

import evolution
from config import Config
from pymoo.algorithms.moo.nsga2 import NSGA2
import os
import evolution, evolution_operations
from pymoo.optimize import minimize

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

config_settings = {
    'n_gen': 1,
    'dataset': 'mnist',
    'pop_size': 2,
    'n_epochs': 1,
    'number_of_objectives': 1}

iterations = 1
iteration_for_second_objective = 2

for i in range(iterations):
    if i == iteration_for_second_objective: config_settings['obj_num'] = 2

    config = Config(config_settings)      
    problem = evolution.EVProblem(config)
    algorithm = NSGA2(pop_size=config.pop_size,
                sampling=evolution_operations.SamplingFromSmall(),
                mutation=evolution_operations.MutationFromSmall(),
                eliminate_duplicates=True)
    res = minimize(problem,
                    algorithm, 
                    callback=evolution.do_every_generations,
                    termination=config.termination)


In [ ]:
''' CNN MNIST digits classification
3-layer CNN for MNIST digits classification 
First 2 layers - Conv2D-ReLU-MaxPool
3rd layer - Conv2D-ReLU-Dropout
4th layer - Dense(10)
Output Activation - softmax
Optimizer - Adam
99.4% test accuracy in 10epochs
https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# input image dimensions
image_size = x_train.shape[1]
# resize and normalize
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
# image is processed as is (square grayscale)
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
pool_size = 2
filters = 64
dropout = 0.4

# model is a stack of CNN-ReLU-MaxPooling
model = Sequential()
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu'))
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu'))
model.add(Flatten())
# dropout added as regularizer
# model.add(Dropout(dropout))
# output layer is 10-dim one-hot vector
print(num_labels)
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

# enable this if pydot can be installed
# pip install pydot
#plot_model(model, to_file='cnn-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of adam optimizer
# accuracy is good metric for classification tasks
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# train the network
model.fit(x_train, y_train, epochs=10, use_multiprocessing=True, batch_size=batch_size)

_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))


In [ ]:
''' CNN MNIST digits classification
3-layer CNN for MNIST digits classification 
First 2 layers - Conv2D-ReLU-MaxPool
3rd layer - Conv2D-ReLU-Dropout
4th layer - Dense(10)
Output Activation - softmax
Optimizer - Adam
99.4% test accuracy in 10epochs
https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

from data_set import DataSet

# load mnist dataset
ds_train, ds_test, ds_info = DataSet('mnist', batch_size=128)

# # compute the number of labels
# num_labels = len(np.unique(y_train))

# # convert to one-hot vector
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# # input image dimensions
# image_size = x_train.shape[1]
# # resize and normalize
# x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
# x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255

# network parameters
# image is processed as is (square grayscale)
# input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
pool_size = 2
filters = 64
dropout = 0.4

# model is a stack of CNN-ReLU-MaxPooling
model = Sequential()
model.add(Conv2D(filters=filters,
                 kernel_size = kernel_size, 
                 activation='relu',
                 input_shape=ds_info.features['image'].shape))
model.add(MaxPooling2D(pool_size))
# model.add(BatchNormalization())
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu'))
model.add(MaxPooling2D(pool_size))
# model.add(BatchNormalization())
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu'))
model.add(Flatten())
# dropout added as regularizer
model.add(Dropout(dropout))
# output layer is 10-dim one-hot vector
model.add(Dense(ds_info.features['label'].num_classes))
model.add(Activation('softmax'))
model.summary()

# enable this if pydot can be installed
# pip install pydot
#plot_model(model, to_file='cnn-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of adam optimizer
# accuracy is good metric for classification tasks
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# train the network
model.fit(ds_train, epochs=10, batch_size=batch_size)

_, acc = model.evaluate(ds_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical, plot_model

from data_set import DataSet

# load mnist dataset
ds_train, ds_test, ds_info = DataSet('cifar10', batch_size=128)

model = Sequential()
model.add(Conv2D(input_shape=ds_info.features['image'].shape, filters=96, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(Conv2D(filters=96, kernel_size=(3,3), strides=2))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(filters=192, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(Conv2D(filters=192, kernel_size=(3,3), strides=2))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(ds_info.features['label'].num_classes, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
n_epochs = 25
batch_size = 128
callbacks_list = None
H = model.fit(ds_train, validation_data=ds_test, 
              epochs=n_epochs, batch_size=batch_size, callbacks=callbacks_list)
print('Done!!!')